# Model building using XGBoost and Aitomatic ETL files

In [1]:
import pandas as pd

## Load and examine the data files

In [2]:
iot_pmfp_data_df = pd.read_feather('https://s3.us-west-1.amazonaws.com/aitomatic.us/pmfp-data/iot_pmfp_data.feather')
iot_pmfp_data_df

,datetime,machineID,volt,rotate,pressure,vibration,model,age,anomaly
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,model3,18,False
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,model3,18,False
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,model3,18,False
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,model3,18,False
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,model3,18,False
...,...,...,...,...,...,...,...,...,...
876095,2016-01-01 02:00:00,100,179.438162,395.222827,102.290715,50.771941,model4,5,False
876096,2016-01-01 03:00:00,100,189.617555,446.207972,98.180607,35.123072,model4,5,False
876097,2016-01-01 04:00:00,100,192.483414,447.816524,94.132837,48.314561,model4,5,False
876098,2016-01-01 05:00:00,100,165.475310,413.771670,104.081073,44.835259,model4,5,False


In [3]:
iot_pmfp_labels_df = pd.read_feather('https://s3.us-west-1.amazonaws.com/aitomatic.us/pmfp-data/iot_pmfp_labels.feather')
iot_pmfp_labels_df

,datetime,machineID,failure_comp1,failure_comp2,failure_comp3,failure_comp4,maint_comp1,maint_comp2,maint_comp3,maint_comp4,error1,error2,error3,error4,error5,failure,maint,error,anomaly
0,2015-01-01 06:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
1,2015-01-01 07:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
2,2015-01-01 08:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
3,2015-01-01 09:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
4,2015-01-01 10:00:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876095,2016-01-01 02:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
876096,2016-01-01 03:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
876097,2016-01-01 04:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False
876098,2016-01-01 05:00:00,100,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False


In [4]:
iot_pmfp_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876100 entries, 0 to 876099
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   datetime   876100 non-null  datetime64[ns]
 1   machineID  876100 non-null  int64         
 2   volt       876100 non-null  float64       
 3   rotate     876100 non-null  float64       
 4   pressure   876100 non-null  float64       
 5   vibration  876100 non-null  float64       
 6   model      876100 non-null  object        
 7   age        876100 non-null  int64         
 8   anomaly    876100 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 54.3+ MB


In [5]:
iot_pmfp_data_df.describe()

,datetime,machineID,volt,rotate,pressure,vibration,age
count,876100,876100.000000,876100.000000,876100.000000,876100.000000,876100.000000,876100.000000
mean,2015-07-02 18:00:00,50.500000,170.777736,446.605119,100.858668,40.385007,11.330000
min,2015-01-01 06:00:00,1.000000,97.333604,138.432075,51.237106,14.877054,0.000000
25%,2015-04-02 12:00:00,25.750000,160.304927,412.305714,93.498181,36.777299,6.750000
50%,2015-07-02 18:00:00,50.500000,170.607338,447.558150,100.425559,40.237247,12.000000
75%,2015-10-02 00:00:00,75.250000,181.004493,482.176600,107.555231,43.784938,16.000000
max,2016-01-01 06:00:00,100.000000,255.124717,695.020984,185.951998,76.791072,20.000000
std,NaN,28.866087,15.509114,52.673886,11.048679,5.370361,5.827619


### "anomaly" column
I wanted to figure out what the **anomaly** column was. Counting the number of anomalies, I found that the anomaly column is the sum of the **error** and **failure** columns.

In [6]:
print(len(iot_pmfp_data_df[iot_pmfp_data_df.anomaly]))
print(len(iot_pmfp_labels_df[(iot_pmfp_labels_df['failure_comp1']==1) | 
                             (iot_pmfp_labels_df['failure_comp2']==1) | 
                             (iot_pmfp_labels_df['failure_comp3']==1) | 
                             (iot_pmfp_labels_df['failure_comp4']==1) | 
                            (iot_pmfp_labels_df['error1']==1) |
                            (iot_pmfp_labels_df['error2']==1) |
                            (iot_pmfp_labels_df['error3']==1) |
                            (iot_pmfp_labels_df['error4']==1) |
                            (iot_pmfp_labels_df['error5']==1)]) )

4332
4332


## Feature engineering

### Telemetry features

#### Mean and standard deviation of telemetry aggregated over 3 hour intervals

In [7]:
# Calculate mean values for telemetry features
temp = []
fields = ['volt', 'rotate', 'pressure', 'vibration']
for col in fields:
    temp.append(pd.pivot_table(iot_pmfp_data_df,
                               index='datetime',
                               columns='machineID',
                               values=col).resample('3H', closed='left', label='right').mean().unstack())
telemetry_mean_3h = pd.concat(temp, axis=1)
telemetry_mean_3h.columns = [i + 'mean_3h' for i in fields]
telemetry_mean_3h.reset_index(inplace=True)

# repeat for standard deviation
temp = []
for col in fields:
    temp.append(pd.pivot_table(iot_pmfp_data_df,
                               index='datetime',
                               columns='machineID',
                               values=col).resample('3H', closed='left', label='right').std().unstack())
telemetry_sd_3h = pd.concat(temp, axis=1)
telemetry_sd_3h.columns = [i + 'sd_3h' for i in fields]
telemetry_sd_3h.reset_index(inplace=True)

telemetry_mean_3h.head()

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h
0,1,2015-01-01 09:00:00,170.028993,449.533798,94.592122,40.893502
1,1,2015-01-01 12:00:00,164.192565,403.949857,105.687417,34.255891
2,1,2015-01-01 15:00:00,168.134445,435.781707,107.793709,41.239405
3,1,2015-01-01 18:00:00,165.514453,430.472823,101.703289,40.373739
4,1,2015-01-01 21:00:00,168.809347,437.111120,90.911060,41.738542


#### Mean and standard deviation of telemetry aggregated over 3 hour intervals rolling over 24 hour windows

In [8]:
temp = []
fields = ['volt', 'rotate', 'pressure', 'vibration']
for col in fields:
    temp.append(pd.pivot_table(iot_pmfp_data_df,index='datetime',
                                               columns='machineID',
                                               values=col).rolling(24).mean().resample('3H',
                                                                                closed='left',
                                                                                label='right').first().unstack())
telemetry_mean_24h = pd.concat(temp, axis=1)
telemetry_mean_24h.columns = [i + 'mean_24h' for i in fields]
telemetry_mean_24h.reset_index(inplace=True)
telemetry_mean_24h = telemetry_mean_24h.loc[-telemetry_mean_24h['voltmean_24h'].isnull()]

# repeat for standard deviation
temp = []
fields = ['volt', 'rotate', 'pressure', 'vibration']
for col in fields:
    temp.append(pd.pivot_table(iot_pmfp_data_df,index='datetime',
                                               columns='machineID',
                                               values=col).rolling(24).std().resample('3H',
                                                                                closed='left',
                                                                                label='right').first().unstack())
telemetry_sd_24h = pd.concat(temp, axis=1)
telemetry_sd_24h.columns = [i + 'sd_24h' for i in fields]
telemetry_sd_24h = telemetry_sd_24h.loc[-telemetry_sd_24h['voltsd_24h'].isnull()]
telemetry_sd_24h.reset_index(inplace=True)

# Notice that a 24h rolling average is not available at the earliest timepoints
telemetry_mean_24h.head(10)

,machineID,datetime,voltmean_24h,rotatemean_24h,pressuremean_24h,vibrationmean_24h
7,1,2015-01-02 06:00:00,169.733809,445.179865,96.797113,40.385160
8,1,2015-01-02 09:00:00,170.614862,446.364859,96.849785,39.736826
9,1,2015-01-02 12:00:00,169.893965,447.009407,97.715600,39.498374
10,1,2015-01-02 15:00:00,171.243444,444.233563,96.666060,40.229370
11,1,2015-01-02 18:00:00,170.792486,448.440437,95.766838,40.055214
12,1,2015-01-02 21:00:00,170.556674,452.267095,98.065860,40.033247
13,1,2015-01-03 00:00:00,168.460525,451.031783,99.273286,38.903462
14,1,2015-01-03 03:00:00,169.772951,447.502464,99.005946,39.389725
15,1,2015-01-03 06:00:00,170.900562,453.864597,100.877342,38.696225
16,1,2015-01-03 09:00:00,169.533156,454.785072,100.050567,39.449734


#### Concatenate the telemetry features into a single dataframe

In [9]:
# merge columns of feature sets created earlier
telemetry_feat = pd.concat([telemetry_mean_3h,
                            telemetry_sd_3h.iloc[:, 2:6],
                            telemetry_mean_24h.iloc[:, 2:6],
                            telemetry_sd_24h.iloc[:, 2:6]], axis=1).dropna()
# add model and age to the machines
telemetry_feat = telemetry_feat.merge(iot_pmfp_data_df[['datetime', 'machineID', 'model', 'age']], on=['datetime', 'machineID'], how='left')
print(telemetry_feat.columns)
telemetry_feat.describe()

Index(['machineID', 'datetime', 'voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'model', 'age'],
      dtype='object')


,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,voltmean_24h,rotatemean_24h,pressuremean_24h,vibrationmean_24h,voltsd_24h,rotatesd_24h,pressuresd_24h,vibrationsd_24h,age
count,290601.000000,290601,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000
mean,50.380935,2015-07-02 22:00:42.478862848,170.774427,446.609386,100.858340,40.383609,13.300173,44.453951,8.885780,4.440575,170.775661,446.609874,100.857574,40.383881,14.919452,49.950788,10.046380,5.002089,11.345226
min,1.000000,2015-01-02 06:00:00,125.532506,211.811184,72.118639,26.569635,0.025509,0.078991,0.027417,0.015278,155.812721,266.010419,91.057429,35.060087,6.380619,18.385248,4.145308,2.144863,0.000000
25%,25.000000,2015-04-03 00:00:00,164.447794,427.564793,96.239534,38.147458,8.028675,26.906319,5.369959,2.684556,168.072275,441.542561,98.669734,39.354077,13.359069,44.669022,8.924165,4.460675,7.000000
50%,50.000000,2015-07-02 21:00:00,170.432407,448.380260,100.235357,40.145874,12.495542,41.793798,8.345801,4.173704,170.212704,449.206885,100.099533,40.072618,14.854186,49.617459,9.921332,4.958793,12.000000
75%,75.000000,2015-10-01 15:00:00,176.610017,468.443933,104.406534,42.226898,17.688520,59.092354,11.789358,5.898512,172.462228,456.366349,101.613047,40.833112,16.395372,54.826993,10.980250,5.484430,16.000000
max,100.000000,2016-01-01 06:00:00,241.420717,586.682904,162.309656,69.311324,58.444332,179.903039,35.659369,18.305595,220.782618,499.096975,152.310351,61.932124,27.664538,103.819404,28.654103,12.325783,20.000000
std,28.798424,NaN,9.498824,33.119738,7.411701,3.475512,6.966389,23.214291,4.656364,2.319989,4.720237,18.070458,4.737293,2.058059,2.261097,7.684305,1.713206,0.799599,5.826345


In [10]:
telemetry_feat

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,voltmean_24h,rotatemean_24h,pressuremean_24h,vibrationmean_24h,voltsd_24h,rotatesd_24h,pressuresd_24h,vibrationsd_24h,model,age
0,1,2015-01-02 06:00:00,180.133784,440.608320,94.137969,41.551544,21.322735,48.770512,2.135684,10.037208,169.733809,445.179865,96.797113,40.385160,15.726970,39.648116,11.904700,5.601191,model3,18
1,1,2015-01-02 09:00:00,176.364293,439.349655,101.553209,36.105580,18.952210,51.329636,13.789279,6.737739,170.614862,446.364859,96.849785,39.736826,15.635083,41.828592,11.326412,5.583521,model3,18
2,1,2015-01-02 12:00:00,160.384568,424.385316,99.598722,36.094637,13.047080,13.702496,9.988609,1.639962,169.893965,447.009407,97.715600,39.498374,13.995465,40.843882,11.036546,5.561553,model3,18
3,1,2015-01-02 15:00:00,170.472461,442.933997,102.380586,40.483002,16.642354,56.290447,3.305739,8.854145,171.243444,444.233563,96.666060,40.229370,13.100364,43.409841,10.972862,6.068674,model3,18
4,1,2015-01-02 18:00:00,163.263806,468.937558,102.726648,40.921802,17.424688,38.680380,9.105775,3.060781,170.792486,448.440437,95.766838,40.055214,13.808489,43.742304,10.988704,7.286129,model3,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290596,100,2015-10-05 09:00:00,188.267556,407.256175,108.931184,36.553233,9.599915,40.722980,1.639521,5.724500,171.826650,441.278667,98.311919,39.196175,16.429023,62.147934,7.475540,5.448962,model4,5
290597,100,2015-10-05 12:00:00,167.859576,465.992407,107.953155,42.708899,14.190347,92.277799,9.577243,0.735339,174.657123,444.147310,98.520388,38.820190,17.019808,64.730136,8.961444,5.833191,model4,5
290598,100,2015-10-05 15:00:00,170.348099,434.234744,104.514343,38.607950,10.232598,49.524471,12.445345,2.596743,173.787879,448.842085,100.028549,39.375067,17.096392,64.718132,9.420879,5.738756,model4,5
290599,100,2015-10-05 18:00:00,152.265370,459.557611,103.536524,40.718426,6.758667,27.051145,12.824247,2.752883,172.496791,442.086577,100.361794,38.943434,15.119775,65.929509,8.836617,6.139142,model4,5


### Error features
Rolling sum of error count over the previous 24 hour window

In [11]:
temp = []
fields = ['error%d' % i for i in range(1,6)]
for col in fields:
    temp.append(pd.pivot_table(iot_pmfp_labels_df,
                                               index='datetime',
                                               columns='machineID',
                                               values=col).rolling(24).sum().resample('3H',
                                                                             closed='left',
                                                                             label='right').first().unstack())
error_count = pd.concat(temp, axis=1)
error_count.columns = [i + 'count' for i in fields]
error_count.reset_index(inplace=True)
error_count = error_count.dropna()
error_count.describe()

,machineID,datetime,error1count,error2count,error3count,error4count,error5count
count,291400.00000,291400,291400.000000,291400.000000,291400.000000,291400.000000,291400.000000
mean,50.50000,2015-07-03 07:30:00,0.027649,0.027069,0.022907,0.019904,0.009753
min,1.00000,2015-01-02 06:00:00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.75000,2015-04-03 06:00:00,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50.50000,2015-07-03 07:30:00,0.000000,0.000000,0.000000,0.000000,0.000000
75%,75.25000,2015-10-02 09:00:00,0.000000,0.000000,0.000000,0.000000,0.000000
max,100.00000,2016-01-01 09:00:00,2.000000,2.000000,2.000000,2.000000,2.000000
std,28.86612,NaN,0.166273,0.164429,0.151453,0.140820,0.098797


In [12]:
error_count

,machineID,datetime,error1count,error2count,error3count,error4count,error5count
7,1,2015-01-02 06:00:00,0.0,0.0,0.0,0.0,0.0
8,1,2015-01-02 09:00:00,0.0,0.0,0.0,0.0,0.0
9,1,2015-01-02 12:00:00,0.0,0.0,0.0,0.0,0.0
10,1,2015-01-02 15:00:00,0.0,0.0,0.0,0.0,0.0
11,1,2015-01-02 18:00:00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
292095,100,2015-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0
292096,100,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0
292097,100,2016-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0
292098,100,2016-01-01 06:00:00,0.0,0.0,0.0,0.0,0.0


### Maintenance features
In the orignal version of this model https://www.kaggle.com/code/d4rklucif3r/predictive-maintenance-anai "number of dyas since the last maintenance" was used as a feature. While this is a good feature, we may not have the record of the last maintenance in the dataset. Authors of the original model simply drop such rows when last maintanace record is not found. I wanted to see if we can use the **maintenance** column as a feature. I will simply count the number of maintenance events that occured within the data available to us. While this is not as powerful as the original feature, it should be OK, since the model is 99% accurate and we can afford to lose some accuracy.

In [13]:
temp = []
fields = ['maint_comp%d' % i for i in range(1,5)]
for col in fields:
    temp.append(pd.pivot_table(iot_pmfp_labels_df,
                                               index='datetime',
                                               columns='machineID',
                                               values=col).expanding().sum().resample('3H',
                                                                             closed='left',
                                                                             label='right').first().unstack())
maint_count = pd.concat(temp, axis=1)
maint_count.columns = [i + '_count' for i in fields]
maint_count.reset_index(inplace=True)
maint_count = maint_count.dropna()
maint_count.describe()

,machineID,datetime,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
count,292100.000000,292100,292100.000000,292100.000000,292100.000000,292100.000000
mean,50.500000,2015-07-02 20:59:59.999999744,3.457254,3.743167,3.440479,3.529966
min,1.000000,2015-01-01 09:00:00,0.000000,0.000000,0.000000,0.000000
25%,25.750000,2015-04-02 15:00:00,1.000000,1.000000,1.000000,1.000000
50%,50.500000,2015-07-02 21:00:00,3.000000,3.000000,3.000000,3.000000
75%,75.250000,2015-10-02 03:00:00,5.000000,6.000000,5.000000,5.000000
max,100.000000,2016-01-01 09:00:00,12.000000,14.000000,14.000000,15.000000
std,28.866119,NaN,2.604536,2.831715,2.563449,2.739109


In [14]:
maint_count

,machineID,datetime,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
0,1,2015-01-01 09:00:00,0.0,0.0,0.0,0.0
1,1,2015-01-01 12:00:00,0.0,0.0,0.0,0.0
2,1,2015-01-01 15:00:00,0.0,0.0,0.0,0.0
3,1,2015-01-01 18:00:00,0.0,0.0,0.0,0.0
4,1,2015-01-01 21:00:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
292095,100,2015-12-31 21:00:00,9.0,3.0,6.0,6.0
292096,100,2016-01-01 00:00:00,9.0,3.0,6.0,6.0
292097,100,2016-01-01 03:00:00,9.0,3.0,6.0,6.0
292098,100,2016-01-01 06:00:00,9.0,3.0,6.0,6.0


### Combine all features into a single dataframe

In [15]:
final_feat = telemetry_feat.merge(error_count, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(maint_count, on=['datetime', 'machineID'], how='inner')

print(final_feat.head())
final_feat.describe()

   machineID            datetime  voltmean_3h  rotatemean_3h  pressuremean_3h   
0          1 2015-01-02 06:00:00   180.133784     440.608320        94.137969  \
1          1 2015-01-02 09:00:00   176.364293     439.349655       101.553209   
2          1 2015-01-02 12:00:00   160.384568     424.385316        99.598722   
3          1 2015-01-02 15:00:00   170.472461     442.933997       102.380586   
4          1 2015-01-02 18:00:00   163.263806     468.937558       102.726648   

   vibrationmean_3h  voltsd_3h  rotatesd_3h  pressuresd_3h  vibrationsd_3h   
0         41.551544  21.322735    48.770512       2.135684       10.037208  \
1         36.105580  18.952210    51.329636      13.789279        6.737739   
2         36.094637  13.047080    13.702496       9.988609        1.639962   
3         40.483002  16.642354    56.290447       3.305739        8.854145   
4         40.921802  17.424688    38.680380       9.105775        3.060781   

   ...  age  error1count  error2count  error

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,...,age,error1count,error2count,error3count,error4count,error5count,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
count,290601.000000,290601,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,...,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000
mean,50.380935,2015-07-02 22:00:42.478862848,170.774427,446.609386,100.858340,40.383609,13.300173,44.453951,8.885780,4.440575,...,11.345226,0.027560,0.027058,0.022846,0.019955,0.009780,3.451041,3.756542,3.441454,3.532727
min,1.000000,2015-01-02 06:00:00,125.532506,211.811184,72.118639,26.569635,0.025509,0.078991,0.027417,0.015278,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,2015-04-03 00:00:00,164.447794,427.564793,96.239534,38.147458,8.028675,26.906319,5.369959,2.684556,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,50.000000,2015-07-02 21:00:00,170.432407,448.380260,100.235357,40.145874,12.495542,41.793798,8.345801,4.173704,...,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000
75%,75.000000,2015-10-01 15:00:00,176.610017,468.443933,104.406534,42.226898,17.688520,59.092354,11.789358,5.898512,...,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,6.000000,5.000000,5.000000
max,100.000000,2016-01-01 06:00:00,241.420717,586.682904,162.309656,69.311324,58.444332,179.903039,35.659369,18.305595,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,12.000000,14.000000,14.000000,15.000000
std,28.798424,NaN,9.498824,33.119738,7.411701,3.475512,6.966389,23.214291,4.656364,2.319989,...,5.826345,0.166026,0.164401,0.151266,0.140998,0.098931,2.590638,2.829210,2.560354,2.737725


In [16]:
final_feat.columns

Index(['machineID', 'datetime', 'voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'model', 'age', 'error1count',
       'error2count', 'error3count', 'error4count', 'error5count',
       'maint_comp1_count', 'maint_comp2_count', 'maint_comp3_count',
       'maint_comp4_count'],
      dtype='object')

### Label construction
**Note** label distribuition is slightly different from `PdM_failures.csv` where clasees look like this:
```python
comp2    259
comp1    192
comp4    179
comp3    131
```
and in this notebook:
```python
failure_comp2       250
failure_comp1       192
failure_comp4       155
failure_comp3       122
```

In [17]:
# import numpy as np
# iot_pmfp_labels_df['none'] = np.where(iot_pmfp_labels_df['failure'] == False, 1, 0)  # 0 is failure, 1 is no failure
# # we convert one-hot encoded columns to a single column of component expected to fail at a given point in time
# iot_pmfp_labels_df['comp_to_fail'] = iot_pmfp_labels_df[['failure_comp1','failure_comp2','failure_comp3','failure_comp4', 'none']].idxmax(axis=1)
# iot_pmfp_labels_df['comp_to_fail'].value_counts()

In [18]:
import numpy as np

iot_pmfp_labels_df['none'] = np.where(iot_pmfp_labels_df['failure'] == False, 1, 0)  # 0 is failure, 1 is no failure
iot_pmfp_labels_df['comp_to_fail'] = iot_pmfp_labels_df[['failure_comp1','failure_comp2','failure_comp3','failure_comp4', 'none']].idxmax(axis=1)
iot_pmfp_labels_df['comp_to_fail'] = np.where(iot_pmfp_labels_df['comp_to_fail'] == 'none', np.nan, iot_pmfp_labels_df['comp_to_fail'])
iot_pmfp_labels_df['comp_to_fail'] = iot_pmfp_labels_df['comp_to_fail'].astype('category')

labeled_features = final_feat.merge(iot_pmfp_labels_df[['datetime', 'machineID', 'comp_to_fail']],
                                                on=['datetime', 'machineID'], how='left')

labeled_features = labeled_features.fillna(method='bfill', limit=7) # fill backward up to 24h
labeled_features['comp_to_fail'] = labeled_features['comp_to_fail'].cat.add_categories(
    'none')
labeled_features['comp_to_fail'] = labeled_features['comp_to_fail'].fillna("none")

In [19]:
labeled_features.describe()

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,...,age,error1count,error2count,error3count,error4count,error5count,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
count,290601.000000,290601,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,...,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000
mean,50.380935,2015-07-02 22:00:42.478862848,170.774427,446.609386,100.858340,40.383609,13.300173,44.453951,8.885780,4.440575,...,11.345226,0.027560,0.027058,0.022846,0.019955,0.009780,3.451041,3.756542,3.441454,3.532727
min,1.000000,2015-01-02 06:00:00,125.532506,211.811184,72.118639,26.569635,0.025509,0.078991,0.027417,0.015278,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,2015-04-03 00:00:00,164.447794,427.564793,96.239534,38.147458,8.028675,26.906319,5.369959,2.684556,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,50.000000,2015-07-02 21:00:00,170.432407,448.380260,100.235357,40.145874,12.495542,41.793798,8.345801,4.173704,...,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000
75%,75.000000,2015-10-01 15:00:00,176.610017,468.443933,104.406534,42.226898,17.688520,59.092354,11.789358,5.898512,...,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,6.000000,5.000000,5.000000
max,100.000000,2016-01-01 06:00:00,241.420717,586.682904,162.309656,69.311324,58.444332,179.903039,35.659369,18.305595,...,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,12.000000,14.000000,14.000000,15.000000
std,28.798424,NaN,9.498824,33.119738,7.411701,3.475512,6.966389,23.214291,4.656364,2.319989,...,5.826345,0.166026,0.164401,0.151266,0.140998,0.098931,2.590638,2.829210,2.560354,2.737725


In [20]:
labeled_features.columns

Index(['machineID', 'datetime', 'voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'model', 'age', 'error1count',
       'error2count', 'error3count', 'error4count', 'error5count',
       'maint_comp1_count', 'maint_comp2_count', 'maint_comp3_count',
       'maint_comp4_count', 'comp_to_fail'],
      dtype='object')

## Model Building using XGBoost and H1st

### XGBoost model abstract class
Inherit from H1st's Model class. In the latest code on https://github.com/h1st-ai/h1st `XGBClassifierModel` is already available. I implemented my own for these reasons:
1. I wanted to use the `XGBClassifier` class from the `xgboost` package instead of the `XGBRegressionModel` with thresholding as implemented in H1st's `XGBClassifierModel`.
2. I wanted to demonstrate skills how to use H1st's `Model` class to build a model.

In [21]:
# Let's look at the distribution of the labels
labeled_features['comp_to_fail'].value_counts()

comp_to_fail
none             284993
failure_comp2      1968
failure_comp1      1464
failure_comp4      1216
failure_comp3       960
Name: count, dtype: int64

In [22]:
import joblib
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

from h1st.model.ml_model import MLModel
from h1st.model.ml_modeler import MLModeler

class XGBClassifierModel(MLModel):
    def predict(self, data: dict) -> dict:
        """Run model inference on incoming data and return predictions in a
           dictionary.
           Input data should have key 'X' with data values
        """
        labelEncoder, model = self.base_model
        predictions = labelEncoder.inverse_transform(model.predict(data['X']))
        return {'predictions': predictions}
    def persist(self, version=0) -> str:
        return joblib.dump(self.base_model, f'xgb_classifier_{version}.pkl')
    def load(self, path: str) -> None:
        self.base_model = joblib.load(path)

class XGBClassifierModeler(MLModeler):
    def __init__(self):
        super().__init__()
        self.model_class = XGBClassifierModel

    def train_base_model(self, prepared_data):
        """trains and returns the base ML model that will be wrapped by the
           H1st MyMLModel
        """
        X, y = prepared_data['X_train'], prepared_data['y_train']
        labelEncoder = LabelEncoder()
        y = labelEncoder.fit_transform(y)
        model = XGBClassifier(random_state=42)
        model.fit(X, y)
        return labelEncoder, model

    def load_data(self):
        """Implementing this function is optional, alternatively data can
           be passed directly to the build_model function. If implemented,
           the build_model function can be run without any input.
        """
        pass

    def  evaluate_model(self, data: dict, ml_model: MLModel) -> dict:
        """Optional, if implemented then metrics will be attached to the
           trained model created by the build_model method, and can be
           persisted along with the model
        """
        x_test = {'X': data['X_test']}
        y_test = data['y_test']
        y_pred = ml_model.predict(x_test)['predictions']
        accuracy = accuracy_score(y_test, y_pred)
        cf = confusion_matrix(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        f1 = 2 * precision * recall / (precision + recall)
        return {'accuracy_score': accuracy,
                'confusion_matrix': cf,
                'recall': recall,
                'precision': precision,
                'f1': f1}

/Users/mike/Documents/Interviews/Aitomatic/H1st/.conda/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/mike/Documents/Interviews/Aitomatic/H1st/.conda/lib/python3.10/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-

### Predictive Maintenance Model
This is the main model that inherits from the `XGBClassifierModel` class. It implements the `load_data` method to use the dataframes created in the previous section. In production code, this method will be using sklearn's Pipeline class to do the feature engineering and label construction.

In [23]:
from sklearn.model_selection import train_test_split
class PredictiveMaintananceModeler(XGBClassifierModeler):
    def __init__(self, train_test_split_ratio=0.3):
        super().__init__()
        self.model_class = XGBClassifierModel
        self.train_test_split_ratio = train_test_split_ratio
    def load_data(self):
        """One-hot encode categorical features and split data into train and test sets
        """
        X = pd.get_dummies(labeled_features.drop(['datetime', 'machineID', 'comp_to_fail'], axis=1))
        y = labeled_features['comp_to_fail']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.train_test_split_ratio, shuffle=False)

        prepared_data = {'X_train': X_train, 'y_train': y_train,
                        'X_test': X_test, 'y_test': y_test}
        return prepared_data

### Model evaluation

In [24]:
my_modeler = PredictiveMaintananceModeler(train_test_split_ratio=0.5)
my_model = my_modeler.build_model()

accuracy = my_model.metrics['accuracy_score'] * 100
print(f"Accuracy (test): {accuracy:.1f}%")
precision_score_value = my_model.metrics['precision'] * 100
print(f"Precision (test): {precision_score_value:.1f}%")
recall_score_value = my_model.metrics['recall'] * 100
print(f"Recall (test): {recall_score_value:.1f}%")
f1_score_value = my_model.metrics['f1'] * 100
print(f"F1 (test): {f1_score_value:.1f}%")

Accuracy (test): 99.9%
Precision (test): 99.9%
Recall (test): 99.9%
F1 (test): 99.9%


### Model persistence and loading
From https://github.com/h1st-ai/h1st/blob/892a8dca71ca4c62ddf02057fdc29abfded69d10/h1st/model/model.py#L56
```
Currently, only sklearn and tensorflow-keras are supported.
```
Therefore I had to use `joblib` package to save and load the model.

In [25]:
my_model.persist()

['xgb_classifier_0.pkl']

In [26]:
my_model.load('xgb_classifier_0.pkl')

In [27]:
inference_date = '2015-06-18 09:00:00'

# Run inference
X_test = pd.get_dummies(labeled_features[labeled_features['datetime'] == inference_date].drop(['datetime', 'machineID', 'comp_to_fail'], axis=1))
y_pred = my_model.predict({'X': X_test})['predictions']
print(f"Predicted component to fail: {y_pred[0]}")
print(f"Actual component to fail: {labeled_features[labeled_features['datetime'] == inference_date]['comp_to_fail'].values[0]}")

Predicted component to fail: failure_comp4
Actual component to fail: failure_comp4


## Pipeline using scikit-learn

### Transformers for feature engineering and label construction
These will live in the `transformers` folder in the project. They will be used in the `load_data` method of the `PredictiveMaintananceModeler` class.

In [28]:
from sklearn.preprocessing import FunctionTransformer

def telemetry_features_3h(iot_pmfp_data_df):
    # Calculate mean values for telemetry features
    temp = []
    fields = ['volt', 'rotate', 'pressure', 'vibration']
    for col in fields:
        temp.append(pd.pivot_table(iot_pmfp_data_df,
                                index='datetime',
                                columns='machineID',
                                values=col).resample('3H', closed='left', label='right').mean().unstack())
    telemetry_mean_3h = pd.concat(temp, axis=1)
    telemetry_mean_3h.columns = [i + 'mean_3h' for i in fields]
    telemetry_mean_3h.reset_index(inplace=True)

    # repeat for standard deviation
    temp = []
    for col in fields:
        temp.append(pd.pivot_table(iot_pmfp_data_df,
                                index='datetime',
                                columns='machineID',
                                values=col).resample('3H', closed='left', label='right').std().unstack())
    telemetry_sd_3h = pd.concat(temp, axis=1)
    telemetry_sd_3h.columns = [i + 'sd_3h' for i in fields]
    telemetry_sd_3h.reset_index(inplace=True)
    telemetry_3h = pd.concat([telemetry_mean_3h, telemetry_sd_3h.iloc[:, 2:6]], axis=1)
    telemetry_3h = telemetry_3h.merge(iot_pmfp_data_df[['datetime', 'machineID', 'model', 'age']], on=['datetime', 'machineID'], how='left')
    return telemetry_3h
transformer_telemetry_features_3h = FunctionTransformer(telemetry_features_3h, validate=False)

def telemetry_features_24h(iot_pmfp_data_df):
    temp = []
    fields = ['volt', 'rotate', 'pressure', 'vibration']
    for col in fields:
        temp.append(pd.pivot_table(iot_pmfp_data_df,index='datetime',
                                                columns='machineID',
                                                values=col).rolling(24).mean().resample('3H',
                                                                                    closed='left',
                                                                                    label='right').first().unstack())
    telemetry_mean_24h = pd.concat(temp, axis=1)
    telemetry_mean_24h.columns = [i + 'mean_24h' for i in fields]
    telemetry_mean_24h.reset_index(inplace=True)
    telemetry_mean_24h = telemetry_mean_24h.loc[-telemetry_mean_24h['voltmean_24h'].isnull()]

    # repeat for standard deviation
    temp = []
    fields = ['volt', 'rotate', 'pressure', 'vibration']
    for col in fields:
        temp.append(pd.pivot_table(iot_pmfp_data_df,index='datetime',
                                                columns='machineID',
                                                values=col).rolling(24).std().resample('3H',
                                                                                    closed='left',
                                                                                    label='right').first().unstack())
    telemetry_sd_24h = pd.concat(temp, axis=1)
    telemetry_sd_24h.columns = [i + 'sd_24h' for i in fields]
    telemetry_sd_24h = telemetry_sd_24h.loc[-telemetry_sd_24h['voltsd_24h'].isnull()]
    telemetry_sd_24h.reset_index(inplace=True)
    return pd.concat([telemetry_mean_24h.iloc[:, 2:6], telemetry_sd_24h.iloc[:, 2:6]], axis=1)
transformer_telemetry_features_24h = FunctionTransformer(telemetry_features_24h, validate=False)

In [29]:
def error_count(iot_pmfp_labels_df):
    temp = []
    fields = ['error%d' % i for i in range(1,6)]
    for col in fields:
        temp.append(pd.pivot_table(iot_pmfp_labels_df,
                                                index='datetime',
                                                columns='machineID',
                                                values=col).rolling(24).sum().resample('3H',
                                                                                closed='left',
                                                                                label='right').first().unstack())
    error_count = pd.concat(temp, axis=1)
    error_count.columns = [i + 'count' for i in fields]
    error_count.reset_index(inplace=True)
    # error_count = error_count.dropna()
    return error_count
transformer_error_count = FunctionTransformer(error_count, validate=False)

def maint_count(iot_pmfp_labels_df):
    temp = []
    fields = ['maint_comp%d' % i for i in range(1,5)]
    for col in fields:
        temp.append(pd.pivot_table(iot_pmfp_labels_df,
                                                index='datetime',
                                                columns='machineID',
                                                values=col).expanding().sum().resample('3H',
                                                                                closed='left',
                                                                                label='right').first().unstack())
    maint_count = pd.concat(temp, axis=1)
    maint_count.columns = [i + '_count' for i in fields]
    maint_count.reset_index(inplace=True)
    # maint_count = maint_count.dropna()
    return maint_count.drop(['datetime', 'machineID'], axis=1)
transformer_maint_count = FunctionTransformer(maint_count, validate=False)

### Chaining transformers into a pipeline
This will be done in the `load_data` method of the `PredictiveMaintananceModeler` class. We use feature transformers and chain them using sklearn's `FeatureUnion` class. We need two pipelines, since we have two input dataframes, one for telemetry and the other for errors.

In [30]:
from sklearn.pipeline import FeatureUnion
telemetry_feat_pipeline = FeatureUnion([
    ('transformer_telemetry_features_3h', transformer_telemetry_features_3h),
    ('transformer_telemetry_features_24h', transformer_telemetry_features_24h),
])
telemetry_feat_pipeline.set_output(transform='pandas')
telemetry_feat = telemetry_feat_pipeline.fit_transform(iot_pmfp_data_df).dropna()
telemetry_feat

With transform="pandas", `func` should return a DataFrame to follow the set_output API.


,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,model,age,voltmean_24h,rotatemean_24h,pressuremean_24h,vibrationmean_24h,voltsd_24h,rotatesd_24h,pressuresd_24h,vibrationsd_24h
7,1,2015-01-02 06:00:00,180.133784,440.608320,94.137969,41.551544,21.322735,48.770512,2.135684,10.037208,model3,18.0,169.733809,445.179865,96.797113,40.385160,15.726970,39.648116,11.904700,5.601191
8,1,2015-01-02 09:00:00,176.364293,439.349655,101.553209,36.105580,18.952210,51.329636,13.789279,6.737739,model3,18.0,170.614862,446.364859,96.849785,39.736826,15.635083,41.828592,11.326412,5.583521
9,1,2015-01-02 12:00:00,160.384568,424.385316,99.598722,36.094637,13.047080,13.702496,9.988609,1.639962,model3,18.0,169.893965,447.009407,97.715600,39.498374,13.995465,40.843882,11.036546,5.561553
10,1,2015-01-02 15:00:00,170.472461,442.933997,102.380586,40.483002,16.642354,56.290447,3.305739,8.854145,model3,18.0,171.243444,444.233563,96.666060,40.229370,13.100364,43.409841,10.972862,6.068674
11,1,2015-01-02 18:00:00,163.263806,468.937558,102.726648,40.921802,17.424688,38.680380,9.105775,3.060781,model3,18.0,170.792486,448.440437,95.766838,40.055214,13.808489,43.742304,10.988704,7.286129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291395,100,2015-10-05 09:00:00,188.267556,407.256175,108.931184,36.553233,9.599915,40.722980,1.639521,5.724500,model4,5.0,171.826650,441.278667,98.311919,39.196175,16.429023,62.147934,7.475540,5.448962
291396,100,2015-10-05 12:00:00,167.859576,465.992407,107.953155,42.708899,14.190347,92.277799,9.577243,0.735339,model4,5.0,174.657123,444.147310,98.520388,38.820190,17.019808,64.730136,8.961444,5.833191
291397,100,2015-10-05 15:00:00,170.348099,434.234744,104.514343,38.607950,10.232598,49.524471,12.445345,2.596743,model4,5.0,173.787879,448.842085,100.028549,39.375067,17.096392,64.718132,9.420879,5.738756
291398,100,2015-10-05 18:00:00,152.265370,459.557611,103.536524,40.718426,6.758667,27.051145,12.824247,2.752883,model4,5.0,172.496791,442.086577,100.361794,38.943434,15.119775,65.929509,8.836617,6.139142


In [31]:
from sklearn.pipeline import FeatureUnion
error_maint_features_pipeline = FeatureUnion([
    ('transformer_error_count', transformer_error_count),
    ('transformer_maint_count', transformer_maint_count),
])
error_maint_features_pipeline.set_output(transform='pandas')
error_maint_features = error_maint_features_pipeline.fit_transform(iot_pmfp_labels_df).dropna()
error_maint_features

With transform="pandas", `func` should return a DataFrame to follow the set_output API.


,machineID,datetime,error1count,error2count,error3count,error4count,error5count,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
7,1,2015-01-02 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,2015-01-02 09:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,2015-01-02 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,1,2015-01-02 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1,2015-01-02 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
292095,100,2015-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,9.0,3.0,6.0,6.0
292096,100,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,9.0,3.0,6.0,6.0
292097,100,2016-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,9.0,3.0,6.0,6.0
292098,100,2016-01-01 06:00:00,0.0,0.0,0.0,0.0,0.0,9.0,3.0,6.0,6.0


### Merge the two pipelines into a single dataframe

In [32]:
final_feat = telemetry_feat.merge(error_maint_features, on=['datetime', 'machineID'], how='left')
final_feat

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,...,vibrationsd_24h,error1count,error2count,error3count,error4count,error5count,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
0,1,2015-01-02 06:00:00,180.133784,440.608320,94.137969,41.551544,21.322735,48.770512,2.135684,10.037208,...,5.601191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2015-01-02 09:00:00,176.364293,439.349655,101.553209,36.105580,18.952210,51.329636,13.789279,6.737739,...,5.583521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2015-01-02 12:00:00,160.384568,424.385316,99.598722,36.094637,13.047080,13.702496,9.988609,1.639962,...,5.561553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2015-01-02 15:00:00,170.472461,442.933997,102.380586,40.483002,16.642354,56.290447,3.305739,8.854145,...,6.068674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,2015-01-02 18:00:00,163.263806,468.937558,102.726648,40.921802,17.424688,38.680380,9.105775,3.060781,...,7.286129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290596,100,2015-10-05 09:00:00,188.267556,407.256175,108.931184,36.553233,9.599915,40.722980,1.639521,5.724500,...,5.448962,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0
290597,100,2015-10-05 12:00:00,167.859576,465.992407,107.953155,42.708899,14.190347,92.277799,9.577243,0.735339,...,5.833191,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0
290598,100,2015-10-05 15:00:00,170.348099,434.234744,104.514343,38.607950,10.232598,49.524471,12.445345,2.596743,...,5.738756,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0
290599,100,2015-10-05 18:00:00,152.265370,459.557611,103.536524,40.718426,6.758667,27.051145,12.824247,2.752883,...,6.139142,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0


### Label construction

In [33]:
def labeled_features_add(data: tuple):
    """Add labels to the data:
    - Add a comp_to_fail column from the failure_comp columns for individual component failures
    - Merge the error and maintenance features
    - Add a none column for no failure
    - When a component marked as failed, backfill all the rows for the last 24 hours. Assumption is that the component has 24 hours to be replaced.
    - Remaining rows with no failure are marked as 'none'
    """
    iot_pmfp_labels_df, final_feat = data
    
    iot_pmfp_labels_df['none'] = np.where(iot_pmfp_labels_df['failure'] == False, 1, 0)  # 0 is failure, 1 is no failure
    iot_pmfp_labels_df['comp_to_fail'] = iot_pmfp_labels_df[['failure_comp1','failure_comp2','failure_comp3','failure_comp4', 'none']].idxmax(axis=1)
    iot_pmfp_labels_df['comp_to_fail'] = np.where(iot_pmfp_labels_df['comp_to_fail'] == 'none', np.nan, iot_pmfp_labels_df['comp_to_fail'])
    iot_pmfp_labels_df['comp_to_fail'] = iot_pmfp_labels_df['comp_to_fail'].astype('category')
    
    labeled_features = final_feat.merge(iot_pmfp_labels_df[['datetime', 'machineID', 'comp_to_fail']],
                                                   on=['datetime', 'machineID'], how='left')
    
    labeled_features = labeled_features.fillna(method='bfill', limit=7) # fill backward up to 24h
    labeled_features['comp_to_fail'] = labeled_features['comp_to_fail'].cat.add_categories(
        'none')
    labeled_features['comp_to_fail'] = labeled_features['comp_to_fail'].fillna("none")
    return labeled_features
transformer_labeled_features = FunctionTransformer(labeled_features_add, validate=False)

In [34]:
labeled_features = transformer_labeled_features.fit_transform((iot_pmfp_labels_df, final_feat)).dropna()
labeled_features

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,...,error1count,error2count,error3count,error4count,error5count,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count,comp_to_fail
0,1,2015-01-02 06:00:00,180.133784,440.608320,94.137969,41.551544,21.322735,48.770512,2.135684,10.037208,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,none
1,1,2015-01-02 09:00:00,176.364293,439.349655,101.553209,36.105580,18.952210,51.329636,13.789279,6.737739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,none
2,1,2015-01-02 12:00:00,160.384568,424.385316,99.598722,36.094637,13.047080,13.702496,9.988609,1.639962,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,none
3,1,2015-01-02 15:00:00,170.472461,442.933997,102.380586,40.483002,16.642354,56.290447,3.305739,8.854145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,none
4,1,2015-01-02 18:00:00,163.263806,468.937558,102.726648,40.921802,17.424688,38.680380,9.105775,3.060781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290596,100,2015-10-05 09:00:00,188.267556,407.256175,108.931184,36.553233,9.599915,40.722980,1.639521,5.724500,...,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0,none
290597,100,2015-10-05 12:00:00,167.859576,465.992407,107.953155,42.708899,14.190347,92.277799,9.577243,0.735339,...,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0,none
290598,100,2015-10-05 15:00:00,170.348099,434.234744,104.514343,38.607950,10.232598,49.524471,12.445345,2.596743,...,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0,none
290599,100,2015-10-05 18:00:00,152.265370,459.557611,103.536524,40.718426,6.758667,27.051145,12.824247,2.752883,...,0.0,0.0,0.0,0.0,0.0,8.0,1.0,5.0,4.0,none


In [35]:
labeled_features['comp_to_fail'].value_counts()

comp_to_fail
none             284993
failure_comp2      1968
failure_comp1      1464
failure_comp4      1216
failure_comp3       960
Name: count, dtype: int64

In [36]:
labeled_features.describe()

,machineID,datetime,voltmean_3h,rotatemean_3h,pressuremean_3h,vibrationmean_3h,voltsd_3h,rotatesd_3h,pressuresd_3h,vibrationsd_3h,...,vibrationsd_24h,error1count,error2count,error3count,error4count,error5count,maint_comp1_count,maint_comp2_count,maint_comp3_count,maint_comp4_count
count,290601.000000,290601,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,...,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000,290601.000000
mean,50.380935,2015-07-02 22:00:42.478862848,170.774427,446.609386,100.858340,40.383609,13.300173,44.453951,8.885780,4.440575,...,5.002089,0.027560,0.027058,0.022846,0.019955,0.009780,3.451041,3.756542,3.441454,3.532727
min,1.000000,2015-01-02 06:00:00,125.532506,211.811184,72.118639,26.569635,0.025509,0.078991,0.027417,0.015278,...,2.144863,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,2015-04-03 00:00:00,164.447794,427.564793,96.239534,38.147458,8.028675,26.906319,5.369959,2.684556,...,4.460675,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
50%,50.000000,2015-07-02 21:00:00,170.432407,448.380260,100.235357,40.145874,12.495542,41.793798,8.345801,4.173704,...,4.958793,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000
75%,75.000000,2015-10-01 15:00:00,176.610017,468.443933,104.406534,42.226898,17.688520,59.092354,11.789358,5.898512,...,5.484430,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,6.000000,5.000000,5.000000
max,100.000000,2016-01-01 06:00:00,241.420717,586.682904,162.309656,69.311324,58.444332,179.903039,35.659369,18.305595,...,12.325783,2.000000,2.000000,2.000000,2.000000,2.000000,12.000000,14.000000,14.000000,15.000000
std,28.798424,NaN,9.498824,33.119738,7.411701,3.475512,6.966389,23.214291,4.656364,2.319989,...,0.799599,0.166026,0.164401,0.151266,0.140998,0.098931,2.590638,2.829210,2.560354,2.737725


### PredictiveMaintananceModeler with sklearn pipeline

In [37]:
# Let's look at the distribution of the labels
labeled_features['comp_to_fail'].value_counts()

comp_to_fail
none             284993
failure_comp2      1968
failure_comp1      1464
failure_comp4      1216
failure_comp3       960
Name: count, dtype: int64

In [38]:
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
class PredictiveMaintananceModeler(XGBClassifierModeler):
    def __init__(self, train_test_split_ratio=0.3):
        super().__init__()
        self.model_class = XGBClassifierModel
        self.train_test_split_ratio = train_test_split_ratio
        self._error_maint_feat_pipeline = FeatureUnion([
            ('transformer_error_count', transformer_error_count),
            ('transformer_maint_count', transformer_maint_count),
        ])
        self._error_maint_feat_pipeline.set_output(transform='pandas')
        self._telemetry_feat_pipeline = FeatureUnion([
            ('transformer_telemetry_features_3h', transformer_telemetry_features_3h),
            ('transformer_telemetry_features_24h', transformer_telemetry_features_24h),
        ])
        self._telemetry_feat_pipeline.set_output(transform='pandas')

    def load_data(self):
        """Generate labeled features and split data into train and test sets
        """
        # load data
        iot_pmfp_data_df = pd.read_feather('https://s3.us-west-1.amazonaws.com/aitomatic.us/pmfp-data/iot_pmfp_data.feather')
        iot_pmfp_labels_df = pd.read_feather('https://s3.us-west-1.amazonaws.com/aitomatic.us/pmfp-data/iot_pmfp_labels.feather')
        
        # add the labels to the error and maint record features
        error_maint_features = self._error_maint_feat_pipeline.fit_transform(iot_pmfp_labels_df).dropna()
        
        # add telemetry features
        telemetry_feat = self._telemetry_feat_pipeline.fit_transform(iot_pmfp_data_df).dropna()

        # merge telemetry and error/maint features into a single dataframe
        final_feat = telemetry_feat.merge(error_maint_features, on=['datetime', 'machineID'], how='left')
        
        # add labels to the data
        labeled_features = transformer_labeled_features.fit_transform((iot_pmfp_labels_df, final_feat)).dropna()
        
        X = pd.get_dummies(labeled_features.drop(['datetime', 'machineID', 'comp_to_fail'], axis=1))
        y = labeled_features['comp_to_fail']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.train_test_split_ratio, shuffle=False)

        prepared_data = {'X_train': X_train, 'y_train': y_train,
                        'X_test': X_test, 'y_test': y_test}
        return prepared_data

In [39]:
my_modeler = PredictiveMaintananceModeler(train_test_split_ratio=0.5)
my_model = my_modeler.build_model()

accuracy = my_model.metrics['accuracy_score'] * 100
print(f"Accuracy (test): {accuracy:.1f}%")
precision_score_value = my_model.metrics['precision'] * 100
print(f"Precision (test): {precision_score_value:.1f}%")
recall_score_value = my_model.metrics['recall'] * 100
print(f"Recall (test): {recall_score_value:.1f}%")
f1_score_value = my_model.metrics['f1'] * 100
print(f"F1 (test): {f1_score_value:.1f}%")

With transform="pandas", `func` should return a DataFrame to follow the set_output API.


Accuracy (test): 99.9%
Precision (test): 99.9%
Recall (test): 99.9%
F1 (test): 99.9%


### Same results as without pipeline!

## Conclusion
We can seee that with simple feature engineering and an XGBoost model, we can achieve 99% accuracy. This is a good starting point for further improvement. The AutoML tools can be used to find the best model and hyperparameter tuning can be used to improve the model further. https://www.kaggle.com/code/d4rklucif3r/predictive-maintenance-anai actually did some AutML and found Gradient Boosting Classifier to be the best model. I did not do any AutoML in this notebook, since the focus was on using H1st to build a model.
In addition, using sklearn's Pipeline class, we can build a pipeline that can be used in production to do the feature engineering and label construction. This pipeline can be used in a web service to make predictions in real time. The pipeline can also be used in a batch job to make predictions on a large dataset. This notebook confirms that the pipeline works as expected.
Next step is to copy the last section of this notebook into a python file and use it in a web service to make predictions in real time and build an application around it.